In [1]:
#!pip install web3

In [2]:
#!pip install multicall

In [3]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [4]:
import time
from datetime import datetime
import requests
import json
import pandas as pd
from web3 import Web3
from web3.exceptions import (ContractLogicError, InvalidEventABI, LogTopicError, MismatchedABI)
from web3._utils.events import get_event_data
from eth_utils import (encode_hex, event_abi_to_log_topic)
from web3.datastructures import AttributeDict
from multicall import Call, Multicall


Infura_Project_ID = 'Infura_Project_ID' # https://infura.io/docs
assert Infura_Project_ID != 'Infura_Project_ID', "Infura Project ID required!"
Etherscan_APIKEY   = None #optional, but recommended
GearboxAddressProvider = '0xcF64698AFF7E5f27A11dff868AF228653ba53be0'

w3_eth = Web3(Web3.HTTPProvider('https://mainnet.infura.io/v3/'+Infura_Project_ID, request_kwargs={'timeout': 10}))
print ('Ethereum connected:', w3_eth.isConnected())


Ethereum connected: True


In [5]:
def parse_abi(abi_dict, abi_type = None):
    recs = []
    for contract_type in abi_dict.keys():
        for rec in [x for x in abi_dict[contract_type] if not abi_type or x['type'] == abi_type]:
            topic = None
            if rec['type'] == 'event':
                topic = encode_hex(event_abi_to_log_topic(rec))
            name = None
            if 'name' in rec:
                name = rec['name']
                
            recs.append({'contract_type': contract_type,
                           'name'        : name,
                           'type'        : rec['type'],
                           'abi'         : rec,
                           'topic'       : topic,
                           })
    
    df = pd.DataFrame(recs)
    return df
def get_logs (w3, contract_address, topics=None, from_block=0, to_block='latest'):
    logs = w3.eth.get_logs({"address": contract_address
                           ,"topics":topics 
                           ,"fromBlock": from_block 
                           ,"toBlock": to_block
                           })

    all_events = []
    for l in logs:
        try:
            evt_topic0 = l['topics'][0].hex()
            evt_abi = df_abi[df_abi['topic']== evt_topic0]['abi'].values[0] 
            evt = get_event_data(w3.codec, evt_abi, l)
        except MismatchedABI: #if for some reason there are other events 
            pass
        all_events.append(evt)
    df = pd.DataFrame(all_events)
    return df

In [6]:
def pull_abi_etherscan(contract_address, apikey = Etherscan_APIKEY):
    url = 'https://api.etherscan.io/api?module=contract&action=getabi'
    params = {'address':contract_address,'apikey' : apikey}
    
    if not apikey:
        time.sleep(5) # rate-limit when apikey is empty
        
    response = requests.get(url, params=params)
    response_json = json.loads(response.text)
    
    if response_json['status']  == '1':
        return json.loads(response_json['result'])
    else:
        raise Exception(response_json['result'])

In [7]:
print(datetime.utcnow(),'start')
AddressProvider = Web3.toChecksumAddress(GearboxAddressProvider)
AddressProvider_abi = pull_abi_etherscan(AddressProvider)

AccountFactory = w3_eth.eth.contract(address=AddressProvider, abi=AddressProvider_abi).functions.getAccountFactory().call()
print('AccountFactory:', AccountFactory)
AccountFactory_abi = pull_abi_etherscan(AccountFactory)
countCreditAccounts = w3_eth.eth.contract(address=AccountFactory, abi=AccountFactory_abi).functions.countCreditAccounts().call()
countCreditAccountsInStock = w3_eth.eth.contract(address=AccountFactory, abi=AccountFactory_abi).functions.countCreditAccountsInStock().call()
print('countCreditAccounts:', countCreditAccounts)
print('countCreditAccountsInStock:', countCreditAccountsInStock)
masterCreditAccount = w3_eth.eth.contract(address=AccountFactory, abi=AccountFactory_abi).functions.masterCreditAccount().call()
print('masterCreditAccount:', masterCreditAccount)
CreditAccount_abi = pull_abi_etherscan(masterCreditAccount)

DataCompressor = w3_eth.eth.contract(address=AddressProvider, abi=AddressProvider_abi).functions.getDataCompressor().call()
print('DataCompressor:', DataCompressor)
DataCompressor_abi = pull_abi_etherscan(DataCompressor)

ContractsRegister = w3_eth.eth.contract(address=AddressProvider, abi=AddressProvider_abi).functions.getContractsRegister().call()
print('ContractsRegister:', ContractsRegister)
ContractsRegister_abi = pull_abi_etherscan(ContractsRegister)

CreditManagers = w3_eth.eth.contract(address=ContractsRegister, abi=ContractsRegister_abi).functions.getCreditManagers().call()
print('CreditManagers:', CreditManagers)

cm_dict = {AccountFactory: {'symbol': None, 'decimals': None}}
allowedTokens = {}
for i, CreditManager in enumerate(CreditManagers):
    print(datetime.utcnow(),'CreditManager ', i+1, 'of', len(CreditManagers))
    if i==0:
        CreditManager_abi = pull_abi_etherscan(CreditManager)

    Token = w3_eth.eth.contract(address=CreditManager, abi=CreditManager_abi).functions.underlyingToken().call()  
    
    if i==0:
        Token_abi = pull_abi_etherscan(Token)   
    
    CreditFilter = w3_eth.eth.contract(address=CreditManager, abi=CreditManager_abi).functions.creditFilter().call()
    
    if i==0:
        CreditFilter_abi = pull_abi_etherscan(CreditFilter)
        
    allowedTokensCount = w3_eth.eth.contract(address=CreditFilter, abi=CreditFilter_abi).functions.allowedTokensCount().call()
    priceOracle        = w3_eth.eth.contract(address=CreditFilter, abi=CreditFilter_abi).functions.priceOracle().call()
    wethAddress        = w3_eth.eth.contract(address=CreditFilter, abi=CreditFilter_abi).functions.wethAddress().call()
    
    if i==0:
        priceOracle_abi = pull_abi_etherscan(priceOracle)
    
    cm_dict[CreditManager] = {'symbol': w3_eth.eth.contract(address=Token, abi=Token_abi).functions.symbol().call(),
                              'decimals': w3_eth.eth.contract(address=Token, abi=Token_abi).functions.decimals().call(),
                              'CreditFilter': CreditFilter,
                              'priceOracle' : priceOracle,
                              'allowedTokensCount': allowedTokensCount,
                              'allowedTokens':{},
                             }
    for token_id in range(allowedTokensCount):
        allowed_token = w3_eth.eth.contract(address=CreditFilter, abi=CreditFilter_abi).functions.allowedTokens(token_id).call()
        allowed_token_symbol = w3_eth.eth.contract(address=allowed_token, abi=Token_abi).functions.symbol().call()
        allowed_token_decimals = w3_eth.eth.contract(address=allowed_token, abi=Token_abi).functions.decimals().call()
        
        try:
            allowed_token_weth_priceOracle = w3_eth.eth.contract(address=priceOracle, abi=priceOracle_abi).functions.getLastPrice(allowed_token, wethAddress).call()
        except ContractLogicError:
            allowed_token_weth_priceOracle = None
            print(allowed_token_symbol+'-WETH getLastPrice error')
        
        try:    
            allowed_token_underlying_priceOracle = w3_eth.eth.contract(address=priceOracle, abi=priceOracle_abi).functions.getLastPrice(allowed_token, Token).call()
        except ContractLogicError:
            allowed_token_underlying_priceOracle = None
            print(allowed_token_symbol+'-WETH getLastPrice error')

        Token_dict={'symbol'          : allowed_token_symbol, 
                    'decimals'        : allowed_token_decimals,
                    'Price_WETH'      : allowed_token_weth_priceOracle,
                    'Price_'+cm_dict[CreditManager]['symbol']: allowed_token_underlying_priceOracle,
                    }
        if allowed_token in allowedTokens:
            allowedTokens[allowed_token].update(Token_dict)
        else:
            allowedTokens[allowed_token] = Token_dict
    
df_abi = parse_abi({'AddressProvider':AddressProvider_abi, 
                    'AccountFactory': AccountFactory_abi, 
                    'DataCompressor': DataCompressor_abi,
                    'CreditManager': CreditManager_abi,
                    'CreditFilter': CreditFilter_abi,
                    'CreditAccount': CreditAccount_abi,
                   })

display(cm_dict) 
display(allowedTokens)

2022-01-08 03:46:13.705998 start
AccountFactory: 0x444CD42BaEdDEB707eeD823f7177b9ABcC779C04
countCreditAccounts: 5001
countCreditAccountsInStock: 4908
masterCreditAccount: 0x373A292b93ff9017D28E64154ef83b99D5C4e270
DataCompressor: 0x0050b1ABD1DD2D9b01ce954E663ff3DbCa9193B1
ContractsRegister: 0xA50d4E7D8946a7c90652339CDBd262c375d54D99
CreditManagers: ['0x777E23A2AcB2fCbB35f6ccF98272d03C722Ba6EB', '0x2664cc24CBAd28749B3Dd6fC97A6B402484De527', '0x968f9a68a98819E2e6Bb910466e191A7b6cf02F0', '0xC38478B0A4bAFE964C3526EEFF534d70E1E09017']
2022-01-08 03:46:48.957187 CreditManager  1 of 4
2022-01-08 03:47:33.219420 CreditManager  2 of 4
2022-01-08 03:47:53.083361 CreditManager  3 of 4
2022-01-08 03:48:12.673530 CreditManager  4 of 4


{'0x444CD42BaEdDEB707eeD823f7177b9ABcC779C04': {'symbol': None,
  'decimals': None},
 '0x777E23A2AcB2fCbB35f6ccF98272d03C722Ba6EB': {'symbol': 'DAI',
  'decimals': 18,
  'CreditFilter': '0x948D33a9537cf13bCC656218B385D19E5b6693e8',
  'priceOracle': '0x0e74a08443c5E39108520589176Ac12EF65AB080',
  'allowedTokensCount': 15,
  'allowedTokens': {}},
 '0x2664cc24CBAd28749B3Dd6fC97A6B402484De527': {'symbol': 'USDC',
  'decimals': 6,
  'CreditFilter': '0x301E7Ed8ac816747A65cf67D8901659e637a4383',
  'priceOracle': '0x0e74a08443c5E39108520589176Ac12EF65AB080',
  'allowedTokensCount': 15,
  'allowedTokens': {}},
 '0x968f9a68a98819E2e6Bb910466e191A7b6cf02F0': {'symbol': 'WETH',
  'decimals': 18,
  'CreditFilter': '0x790D170C12e62aDa6CD2D409a50177680c4ddF29',
  'priceOracle': '0x0e74a08443c5E39108520589176Ac12EF65AB080',
  'allowedTokensCount': 15,
  'allowedTokens': {}},
 '0xC38478B0A4bAFE964C3526EEFF534d70E1E09017': {'symbol': 'WBTC',
  'decimals': 8,
  'CreditFilter': '0xcF223eB26dA2Bf147D01b750

{'0x6B175474E89094C44Da98b954EedeAC495271d0F': {'symbol': 'DAI',
  'decimals': 18,
  'Price_WETH': 310096377636264,
  'Price_DAI': 1000000000000000000,
  'Price_USDC': 993284147212080526,
  'Price_WBTC': 23499419560308},
 '0x111111111117dC0aa78b770fA6A738034120C302': {'symbol': '1INCH',
  'decimals': 18,
  'Price_WETH': 680554020000000,
  'Price_DAI': 2194653240349277446,
  'Price_USDC': 2179914272266561244,
  'Price_WBTC': 51573077284357},
 '0x7Fc66500c84A76Ad7e9c93437bFc5Ac33E2DDaE9': {'symbol': 'AAVE',
  'decimals': 18,
  'Price_WETH': 68800000000000000,
  'Price_DAI': 221866506550104998732,
  'Price_USDC': 220376483753544521911,
  'Price_WBTC': 5213734123800761},
 '0xc00e94Cb662C3520282E6f5717214004A7f26888': {'symbol': 'COMP',
  'decimals': 18,
  'Price_WETH': 60700000000000004,
  'Price_DAI': 195745595168479277768,
  'Price_USDC': 194430996567444089557,
  'Price_WBTC': 4599907867946312},
 '0x1494CA1F11D487c2bBe4543E90080AeBa4BA3C2b': {'symbol': 'DPI',
  'decimals': 18,
  'Price_W

In [8]:
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]
def combine_description(description):
    text_description = ''
    for x in description:
        if x['type']=='tuple': 
            text_description += '(' + combine_description(x['components'])+')'+','
        elif x['type']=='tuple[]': 
            text_description += '(' + combine_description(x['components']) + ')[]' +','
            
        else:
            text_description += x['internalType']+',' 
    return text_description[0:-1]      

def get_function_signature(function_name, df_abi, inputs=None, outputs=None):
    if not inputs:
        inputs = df_abi[df_abi['name']==function_name]['abi'].values[0]['inputs']
    if not outputs:
        outputs= df_abi[df_abi['name']==function_name]['abi'].values[0]['outputs']
    
    return function_name +'(' + combine_description(inputs) + ')' + '(' + combine_description(outputs) +')' 

def get_data_multicall(df, function_name, df_abi, contract_address = None):
    
    function_signature = get_function_signature(function_name, df_abi )
    #print(function_signature)
    
    if function_name == 'creditAccounts':
        multi_result = Multicall([
                    Call(contract_address, [function_signature, x], [[x, Web3.toChecksumAddress]]) for x in df['id']
                    ]
                    ,_w3 = w3_eth)
    elif function_name == 'creditManager':
        multi_result = Multicall([
                        Call(y, [function_signature], [[x, Web3.toChecksumAddress]]) for x,y in zip(df['id'],df['CA'])
                        ]
                        ,_w3 = w3_eth) 
    elif function_name == 'borrowedAmount':
        multi_result = Multicall([
                        Call(y, [function_signature], [[x, None]]) for x,y in zip(df['id'],df['CA'])
                        ]
                        ,_w3 = w3_eth) 
    elif function_name == 'since':
        multi_result = Multicall([
                        Call(y, [function_signature], [[x, None]]) for x,y in zip(df['id'],df['CA'])
                        ]
                        ,_w3 = w3_eth)
    elif function_name == 'getCreditAccountDataExtended':
        #print(function_signature)
        multi_result = Multicall([
                        Call(contract_address, [function_signature, y, z], [[x, None]]) for x,y,z in zip(df['id'],df['CM'],df['Borrower'])
                        ]
                        ,_w3 = w3_eth)    
    try:
        multi_result = multi_result()
    except (requests.exceptions.HTTPError):
        time.sleep(3)
        multi_result = multi_result()
        
    d_multi_result = AttributeDict.recursive(multi_result)
    return d_multi_result


In [9]:
df = pd.DataFrame()
df['id'] = range(countCreditAccounts)
for ids in list(chunks(list(df['id']), 400)): #chunk size for multicall = 400
    id_range = list(df['id'].isin(ids))
    d_ca = get_data_multicall(df.loc[id_range], 'creditAccounts', df_abi, AccountFactory)
    df.loc[id_range,'CA'] = df.loc[id_range].apply(lambda x: d_ca[x['id']], axis=1)

    d_cm = get_data_multicall(df.loc[id_range], 'creditManager', df_abi)
    df.loc[id_range,'CM'] = df.loc[id_range].apply(lambda x: d_cm[x['id']], axis=1)
    
    d_amount = get_data_multicall(df.loc[id_range], 'borrowedAmount', df_abi)
    df.loc[id_range,'borrowedAmount'] = df.loc[id_range].apply(lambda x: d_amount[x['id']] if d_amount[x['id']] > 1 else 0 , axis=1)
    
    d_since = get_data_multicall(df.loc[id_range], 'since', df_abi)
    df.loc[id_range,'Since'] = df.loc[id_range].apply(lambda x: d_since[x['id']], axis=1)

df['Since'] = df['Since'].astype(int)
df['Decimals'] = df.apply(lambda x: cm_dict[x['CM']]['decimals'], axis=1)
df['Symbol'] = df.apply(lambda x: cm_dict[x['CM']]['symbol'] , axis=1)
df

,id,CA,CM,borrowedAmount,Since,Decimals,Symbol
0,0,0xb5DE854F7Db3164c8F9eFeFFED4c06317BCdbBF1,0x777E23A2AcB2fCbB35f6ccF98272d03C722Ba6EB,2000000000000000000000,13858003,18.0,DAI
1,1,0xA3F1E5d5fb80B3bB0F1b04819F0930C4E0f32AF1,0x968f9a68a98819E2e6Bb910466e191A7b6cf02F0,1250000000000000000,13862947,18.0,WETH
2,2,0xe541B88d68602E5f3Fb511633bAc708BACD8EA4c,0x777E23A2AcB2fCbB35f6ccF98272d03C722Ba6EB,3000000000000000000000,13862953,18.0,DAI
3,3,0xC581Ff1a21f42B315DEDbE219a7Ed2B0fA47aBd5,0x968f9a68a98819E2e6Bb910466e191A7b6cf02F0,900000000000000000,13862987,18.0,WETH
4,4,0x5Ce9C22aC551d3b72136B3fe446902B1af0f3654,0x968f9a68a98819E2e6Bb910466e191A7b6cf02F0,854784188060446794,13864162,18.0,WETH
...,...,...,...,...,...,...,...
4996,4996,0x7E0177E914e3Cf5B69067874d4316BF552037eB2,0x444CD42BaEdDEB707eeD823f7177b9ABcC779C04,0,13819220,NaN,None
4997,4997,0xD6Cdc10FB4eCf7201e91488Ae64C4AD55e6c2515,0x444CD42BaEdDEB707eeD823f7177b9ABcC779C04,0,13819221,NaN,None
4998,4998,0xD1168F931863CCcEf0102041f275cBA5C49907a2,0x444CD42BaEdDEB707eeD823f7177b9ABcC779C04,0,13819222,NaN,None
4999,4999,0xee6B3a0B5a750902c4513508bef90963e88F1c80,0x444CD42BaEdDEB707eeD823f7177b9ABcC779C04,0,13819222,NaN,None


In [10]:
#Open, Close, Repay, Liquidate
OpenCreditAccount_topic      =  df_abi[(df_abi['name']=='OpenCreditAccount') &(df_abi['type']=='event')]['topic'].values[0]
CloseCreditAccount_topic     =  df_abi[(df_abi['name']=='CloseCreditAccount')&(df_abi['type']=='event')]['topic'].values[0]
RepayCreditAccount_topic     =  df_abi[(df_abi['name']=='RepayCreditAccount')&(df_abi['type']=='event')]['topic'].values[0]
LiquidateCreditAccount_topic =  df_abi[(df_abi['name']=='LiquidateCreditAccount')&(df_abi['type']=='event')]['topic'].values[0]

print('OpenCreditAccount_topic:', OpenCreditAccount_topic)
print('CloseCreditAccount_topic:', CloseCreditAccount_topic)
print('RepayCreditAccount_topic:', RepayCreditAccount_topic)
print('LiquidateCreditAccount_topic:', LiquidateCreditAccount_topic)

i=0
for row in df.loc[df['CM']!=AccountFactory].itertuples():
    i+=1
    logs = get_logs(w3_eth, row.CM, [[OpenCreditAccount_topic, 
                                      CloseCreditAccount_topic, 
                                      RepayCreditAccount_topic, 
                                      LiquidateCreditAccount_topic]]
                    , row.Since, 'latest')
    
    open_events = logs[logs['event'] =='OpenCreditAccount']['args'].values 
    close_events = logs[logs['event'] =='CloseCreditAccount']['args'].values 
    repay_events = logs[logs['event'] =='RepayCreditAccount']['args'].values
    liquidate_event = logs[logs['event'] =='LiquidateCreditAccount']['args'].values
    
    CA_open_event = [x for x in open_events if x['creditAccount']== row.CA] # Open
    if len(CA_open_event) > 0:
        Borrower = CA_open_event[0]['onBehalfOf']
        df.loc[df['id']==row.id, 'Borrower'] = Borrower
    
        CA_close_event = [x for x in close_events if x['to']== Borrower] # Close
        if len(CA_close_event) > 0:
            df.loc[df['id']==row.id, ['Borrower', 'borrowedAmount']] = [None, 0]

        CA_repay_event = [x for x in repay_events if x['to']== Borrower] # Repay
        if len(CA_repay_event) > 0:
            df.loc[df['id']==row.id, ['Borrower', 'borrowedAmount']] = [None, 0]

        CA_liquidate_event = [x for x in liquidate_event if x['owner']== Borrower] # Liquidate
        if len(CA_liquidate_event) > 0:
            df.loc[df['id']==row.id, ['Borrower', 'borrowedAmount']] = [None, 0]
            
    if i % 50 == 0:
        print (datetime.utcnow(), ':', i)

print(datetime.utcnow(), i, 'end')
display(df)    


OpenCreditAccount_topic: 0x7b20ae77867a263a1074203a2da261ef0d096c99395c59c9d4a0104b9f334a27
CloseCreditAccount_topic: 0xca05b632388199c23de1352b2e96fd72a0ec71611683330b38060c004bbf0a76
RepayCreditAccount_topic: 0xe7c7987373a0cc4913d307f23ab8ef02e0333a2af445065e2ef7636cffc6daa7
LiquidateCreditAccount_topic: 0x5e5da6c348e62989f9cfe029252433fc99009b7d28fa3c20d675520a10ff5896
2022-01-08 03:49:48.831737 : 50
2022-01-08 03:50:05.083985 : 100
2022-01-08 03:50:19.597016 : 150
2022-01-08 03:50:28.188437 183 end


,id,CA,CM,borrowedAmount,Since,Decimals,Symbol,Borrower
0,0,0xb5DE854F7Db3164c8F9eFeFFED4c06317BCdbBF1,0x777E23A2AcB2fCbB35f6ccF98272d03C722Ba6EB,0,13858003,18.0,DAI,None
1,1,0xA3F1E5d5fb80B3bB0F1b04819F0930C4E0f32AF1,0x968f9a68a98819E2e6Bb910466e191A7b6cf02F0,0,13862947,18.0,WETH,None
2,2,0xe541B88d68602E5f3Fb511633bAc708BACD8EA4c,0x777E23A2AcB2fCbB35f6ccF98272d03C722Ba6EB,0,13862953,18.0,DAI,None
3,3,0xC581Ff1a21f42B315DEDbE219a7Ed2B0fA47aBd5,0x968f9a68a98819E2e6Bb910466e191A7b6cf02F0,0,13862987,18.0,WETH,None
4,4,0x5Ce9C22aC551d3b72136B3fe446902B1af0f3654,0x968f9a68a98819E2e6Bb910466e191A7b6cf02F0,854784188060446794,13864162,18.0,WETH,0x53829d517D8fA7D59d3D40E20251e519C079985F
...,...,...,...,...,...,...,...,...
4996,4996,0x7E0177E914e3Cf5B69067874d4316BF552037eB2,0x444CD42BaEdDEB707eeD823f7177b9ABcC779C04,0,13819220,NaN,None,NaN
4997,4997,0xD6Cdc10FB4eCf7201e91488Ae64C4AD55e6c2515,0x444CD42BaEdDEB707eeD823f7177b9ABcC779C04,0,13819221,NaN,None,NaN
4998,4998,0xD1168F931863CCcEf0102041f275cBA5C49907a2,0x444CD42BaEdDEB707eeD823f7177b9ABcC779C04,0,13819222,NaN,None,NaN
4999,4999,0xee6B3a0B5a750902c4513508bef90963e88F1c80,0x444CD42BaEdDEB707eeD823f7177b9ABcC779C04,0,13819222,NaN,None,NaN


In [11]:
def getCreditAccountDataExtended(id, cm, borrower):
    try:
        return w3_eth.eth.contract(address=DataCompressor, abi=DataCompressor_abi).functions.getCreditAccountDataExtended(cm, borrower).call()
    except:
        print('Error: id=',id,'CM=',cm,'Borrower=',borrower)


data_cols = [x['name'] for x in df_abi[df_abi['name']=='getCreditAccountDataExtended']['abi'].values[0]['outputs'][0]['components']]

batchtime = datetime.utcnow()
df['batchtime'] = batchtime

def get_token_balance(df_row, data_cols, d_data):
    ret = 0
    try:
        if d_data[df_row['id']]: 
            ret = {allowedTokens[Web3.toChecksumAddress(d[0])]['symbol']:d[1] for d in list({y:z for y, z in zip(data_cols, d_data[df_row['id']])}['balances'])}[token]
    except KeyError:
        pass
    return ret

for ids in list(chunks(list(df[pd.notna(df['Borrower'])].loc[:,'id']), 400)): #chunk size for multicall = 400
    id_range = list(df['id'].isin(ids))
    try:
        d_data = get_data_multicall(df.loc[id_range], 'getCreditAccountDataExtended', df_abi, DataCompressor)
    except ContractLogicError:
        print('getCreditAccountDataExtended: Multicall error, calling it one by one')
        d_data = {x:getCreditAccountDataExtended(x,y,z) for x,y,z in zip(df.loc[id_range]['id'],df.loc[id_range]['CM'],df.loc[id_range]['Borrower'])}
        d_data = AttributeDict.recursive(d_data)
    for data in data_cols:
        if data not in ['balances', 'inUse', 'borrower', 'addr', 'borrower', 'creditManager', 'since']: #duplicate columns
            df.loc[id_range, data] = df.loc[id_range].apply(lambda x: {y:z for y, z in zip(data_cols, d_data[x['id']])}[data] if d_data[x['id']] else None
                                                            , axis=1)
        
    tokens_to_frame = [allowedTokens[x]['symbol'] for x in allowedTokens] 
    for token in tokens_to_frame:
        df.loc[id_range, 'Balance_'+token] = df.loc[id_range].apply(lambda x: get_token_balance(x, data_cols, d_data)
                                                        , axis=1)
    
        
df 


,id,CA,CM,borrowedAmount,Since,Decimals,Symbol,Borrower,batchtime,underlyingToken,...,Balance_FEI,Balance_LINK,Balance_SNX,Balance_UNI,Balance_USDC,Balance_WBTC,Balance_WETH,Balance_YFI,Balance_yvDAI,Balance_yvUSDC
0,0,0xb5DE854F7Db3164c8F9eFeFFED4c06317BCdbBF1,0x777E23A2AcB2fCbB35f6ccF98272d03C722Ba6EB,0,13858003,18.0,DAI,None,2022-01-08 03:50:28.217642,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,0xA3F1E5d5fb80B3bB0F1b04819F0930C4E0f32AF1,0x968f9a68a98819E2e6Bb910466e191A7b6cf02F0,0,13862947,18.0,WETH,None,2022-01-08 03:50:28.217642,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,0xe541B88d68602E5f3Fb511633bAc708BACD8EA4c,0x777E23A2AcB2fCbB35f6ccF98272d03C722Ba6EB,0,13862953,18.0,DAI,None,2022-01-08 03:50:28.217642,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,0xC581Ff1a21f42B315DEDbE219a7Ed2B0fA47aBd5,0x968f9a68a98819E2e6Bb910466e191A7b6cf02F0,0,13862987,18.0,WETH,None,2022-01-08 03:50:28.217642,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,0x5Ce9C22aC551d3b72136B3fe446902B1af0f3654,0x968f9a68a98819E2e6Bb910466e191A7b6cf02F0,854784188060446794,13864162,18.0,WETH,0x53829d517D8fA7D59d3D40E20251e519C079985F,2022-01-08 03:50:28.217642,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,...,0,0,0,0,0.0,0.0,1.154784e+18,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4996,4996,0x7E0177E914e3Cf5B69067874d4316BF552037eB2,0x444CD42BaEdDEB707eeD823f7177b9ABcC779C04,0,13819220,NaN,None,None,2022-01-08 03:50:28.217642,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4997,4997,0xD6Cdc10FB4eCf7201e91488Ae64C4AD55e6c2515,0x444CD42BaEdDEB707eeD823f7177b9ABcC779C04,0,13819221,NaN,None,None,2022-01-08 03:50:28.217642,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4998,4998,0xD1168F931863CCcEf0102041f275cBA5C49907a2,0x444CD42BaEdDEB707eeD823f7177b9ABcC779C04,0,13819222,NaN,None,None,2022-01-08 03:50:28.217642,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4999,4999,0xee6B3a0B5a750902c4513508bef90963e88F1c80,0x444CD42BaEdDEB707eeD823f7177b9ABcC779C04,0,13819222,NaN,None,None,2022-01-08 03:50:28.217642,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
df[pd.notna(df['Borrower'])] # active CAs

,id,CA,CM,borrowedAmount,Since,Decimals,Symbol,Borrower,batchtime,underlyingToken,...,Balance_FEI,Balance_LINK,Balance_SNX,Balance_UNI,Balance_USDC,Balance_WBTC,Balance_WETH,Balance_YFI,Balance_yvDAI,Balance_yvUSDC
4,4,0x5Ce9C22aC551d3b72136B3fe446902B1af0f3654,0x968f9a68a98819E2e6Bb910466e191A7b6cf02F0,854784188060446794,13864162,18.0,WETH,0x53829d517D8fA7D59d3D40E20251e519C079985F,2022-01-08 03:50:28.217642,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,...,0,0,0,0,0.000000e+00,0.0,1.154784e+18,0.0,0.0,0.0
8,8,0xF91027C04807c71c7771a2314A622dFA0fE509A1,0x968f9a68a98819E2e6Bb910466e191A7b6cf02F0,1000000000000000000,13864789,18.0,WETH,0x896b94f4f27f12369698C302e2049cAe86936BbB,2022-01-08 03:50:28.217642,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,...,0,0,0,0,0.000000e+00,0.0,2.000000e+18,0.0,0.0,0.0
11,11,0x0B9851B7bcE408dd36d3473a0A0eE8F14ce64ca3,0x968f9a68a98819E2e6Bb910466e191A7b6cf02F0,290000000000000000,13864924,18.0,WETH,0xC16414AC1fedfDAC4F8A09674D994e1BbB9d7113,2022-01-08 03:50:28.217642,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,...,0,0,0,0,0.000000e+00,0.0,1.290000e+18,0.0,0.0,0.0
14,14,0x654C0aDcA189042B16A849d2fF2ED476F525c39e,0x777E23A2AcB2fCbB35f6ccF98272d03C722Ba6EB,15000000000000000000000,13865587,18.0,DAI,0x3aB0818Ba16175D5a41a4076De227c8a55AD6038,2022-01-08 03:50:28.217642,0x6b175474e89094c44da98b954eedeac495271d0f,...,0,0,0,0,0.000000e+00,0.0,0.000000e+00,0.0,0.0,0.0
15,15,0x01782032De554e86badFC84aB877e5f0eB18dC92,0x968f9a68a98819E2e6Bb910466e191A7b6cf02F0,500000000000000000,13865617,18.0,WETH,0x3aB0818Ba16175D5a41a4076De227c8a55AD6038,2022-01-08 03:50:28.217642,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,...,0,0,0,0,0.000000e+00,0.0,1.000000e+18,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
175,175,0x51BedfB7050a2e7345839F73E280Af0f32304Ca2,0x777E23A2AcB2fCbB35f6ccF98272d03C722Ba6EB,16242186332396047929966,13951340,18.0,DAI,0x176E2042a510d00aa69aA02E5f24D438CC6d8aB2,2022-01-08 03:50:28.217642,0x6b175474e89094c44da98b954eedeac495271d0f,...,0,0,0,0,0.000000e+00,0.0,4.616364e+18,0.0,0.0,0.0
176,176,0x91C9e43436bc149c1fc8f2D1Ef56953653b0f70e,0x2664cc24CBAd28749B3Dd6fC97A6B402484De527,3000000000,13951363,6.0,USDC,0x6b4B48CCDb446A109AE07D8b027CE521B5e2F1Ff,2022-01-08 03:50:28.217642,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,...,0,0,0,0,4.000000e+09,0.0,0.000000e+00,0.0,0.0,0.0
178,178,0x05aba8736Da766956ecc683a9a90CF9780341C59,0x2664cc24CBAd28749B3Dd6fC97A6B402484De527,20100000000,13952138,6.0,USDC,0x8F8B4759dC93CA55bD6997DF719F20F581F10F5C,2022-01-08 03:50:28.217642,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,...,0,0,0,0,0.000000e+00,0.0,8.780923e+18,0.0,0.0,0.0
181,181,0xAF0Cf890F157e092D97961308225423011c8eb6E,0x2664cc24CBAd28749B3Dd6fC97A6B402484De527,19999999998,13953968,6.0,USDC,0x9BDf1B88Cf1e5620a12D78307E4B3613d87dB717,2022-01-08 03:50:28.217642,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,...,0,0,0,0,3.000000e+10,0.0,0.000000e+00,0.0,0.0,0.0


In [13]:
print('batchtime', batchtime)
print('countCreditAccounts - countCreditAccountsInStock = ', countCreditAccounts - countCreditAccountsInStock)

batchtime 2022-01-08 03:50:28.217642
countCreditAccounts - countCreditAccountsInStock =  93


In [14]:
#For compatability with BQ data types 
numeric_cols = [x for x in df.columns if x not in ['CA', 'CM' ,'Symbol', 'Borrower', 'batchtime', 
                                                      'underlyingToken', 'canBeClosed']]
df[numeric_cols] = df[numeric_cols].astype('float64')
df['canBeClosed'] =  df['canBeClosed'].astype('bool')

df.dtypes

id                                   float64
CA                                    object
CM                                    object
borrowedAmount                       float64
Since                                float64
Decimals                             float64
Symbol                                object
Borrower                              object
batchtime                     datetime64[ns]
underlyingToken                       object
borrowedAmountPlusInterest           float64
totalValue                           float64
healthFactor                         float64
borrowRate                           float64
repayAmount                          float64
liquidationAmount                    float64
canBeClosed                             bool
cumulativeIndexAtOpen                float64
Balance_DAI                          float64
Balance_1INCH                        float64
Balance_AAVE                         float64
Balance_COMP                         float64
Balance_DP

In [15]:
df_price_oracle = pd.DataFrame.from_dict([allowedTokens[x] for x in allowedTokens])
df_price_oracle = df_price_oracle.set_index('symbol', drop=True).transpose()
df_price_oracle.columns = ['Price_'+x for x in df_price_oracle.columns]
df_price_oracle[[x.replace('Price','Decimals') for x in df_price_oracle.columns]] = df_price_oracle.loc['decimals',:]
df_price_oracle = df_price_oracle.drop(index = 'decimals')

df_price_oracle = df_price_oracle.reset_index()
df_price_oracle = df_price_oracle.rename(columns={'index': 'Price_Asset'})
df_price_oracle['Price_Asset'] = df_price_oracle['Price_Asset'].apply(lambda x: x.replace('Price_',''))

df_price_oracle['batchtime'] = batchtime

#For compatability with BQ data types 
numeric_cols = [x for x in df_price_oracle.columns if x not in ['Price_Asset', 'batchtime']]
df_price_oracle[numeric_cols] = df_price_oracle[numeric_cols].astype('float64')

df_price_oracle

,Price_Asset,Price_DAI,Price_1INCH,Price_AAVE,Price_COMP,Price_DPI,Price_FEI,Price_LINK,Price_SNX,Price_UNI,...,Decimals_LINK,Decimals_SNX,Decimals_UNI,Decimals_USDC,Decimals_WBTC,Decimals_WETH,Decimals_YFI,Decimals_yvDAI,Decimals_yvUSDC,batchtime
0,WETH,3.100964e+14,6.805540e+14,6.880000e+16,6.070000e+16,7.965134e+16,3.133214e+14,8.264669e+15,1.721209e+15,4.926410e+15,...,18.0,18.0,18.0,6.0,8.0,18.0,18.0,18.0,6.0,2022-01-08 03:50:28.217642
1,DAI,1.000000e+18,2.194653e+18,2.218665e+20,1.957456e+20,2.568599e+20,1.010400e+18,2.665194e+19,5.550561e+18,1.588671e+19,...,18.0,18.0,18.0,6.0,8.0,18.0,18.0,18.0,6.0,2022-01-08 03:50:28.217642
2,USDC,9.932841e+17,2.179914e+18,2.203765e+20,1.944310e+20,2.551349e+20,1.003614e+18,2.647295e+19,5.513285e+18,1.578001e+19,...,18.0,18.0,18.0,6.0,8.0,18.0,18.0,18.0,6.0,2022-01-08 03:50:28.217642
3,WBTC,2.349942e+13,5.157308e+13,5.213734e+15,4.599908e+15,6.036059e+15,2.374381e+13,6.263051e+14,1.304350e+14,3.733284e+14,...,18.0,18.0,18.0,6.0,8.0,18.0,18.0,18.0,6.0,2022-01-08 03:50:28.217642


**Write data frames to GCP BigQuery**

In [16]:
#!pip install pandas_gbq

In [17]:
from google.oauth2 import service_account
import pandas_gbq
credentials = service_account.Credentials.from_service_account_file(
    'gearbox-336415-5ed144668529.json',
)
gcp_project_id = 'gearbox-336415'

pandas_gbq.to_gbq(df, 
                  'gearbox.credit_account', 
                  project_id=gcp_project_id, 
                  if_exists = 'append', 
                  progress_bar = False)
print(datetime.utcnow(), 'gearbox.credit_account, insert done')

pandas_gbq.to_gbq(df_price_oracle, 
                  'gearbox.price_oracle', 
                  project_id=gcp_project_id, 
                  if_exists = 'append', 
                  progress_bar = False)
print(datetime.utcnow(), 'gearbox.price_oracle, insert done')

2022-01-08 03:50:47.499769 gearbox.credit_account, insert done
2022-01-08 03:50:54.972661 gearbox.price_oracle, insert done


In [18]:
#Open, Close, Repay, Liquidate, AddCollateral, IncreaseBorrowedAmount
OpenCreditAccount_topic      =  df_abi[(df_abi['name']=='OpenCreditAccount') &(df_abi['type']=='event')]['topic'].values[0]
CloseCreditAccount_topic     =  df_abi[(df_abi['name']=='CloseCreditAccount')&(df_abi['type']=='event')]['topic'].values[0]
RepayCreditAccount_topic     =  df_abi[(df_abi['name']=='RepayCreditAccount')&(df_abi['type']=='event')]['topic'].values[0]
LiquidateCreditAccount_topic =  df_abi[(df_abi['name']=='LiquidateCreditAccount')&(df_abi['type']=='event')]['topic'].values[0]
AddCollateral_topic          =  df_abi[(df_abi['name']=='AddCollateral')&(df_abi['type']=='event')]['topic'].values[0]
IncreaseBorrowedAmount_topic =  df_abi[(df_abi['name']=='IncreaseBorrowedAmount')&(df_abi['type']=='event')]['topic'].values[0]



try:
    df_blocknum = pandas_gbq.read_gbq('select max(blockNumber) from gearbox.account_events', 
                             project_id=gcp_project_id,
                             progress_bar_type = None,)
    from_block = int(df_blocknum.iloc[0,0]) + 1
except pandas_gbq.exceptions.GenericGBQException:
    print('The table does not exist?')
    from_block = 0
    
print(datetime.utcnow(), 'from_block=', from_block)

df_events = pd.DataFrame()

for CM in CreditManagers:
    logs = get_logs(w3_eth, CM, [[OpenCreditAccount_topic, 
                                      CloseCreditAccount_topic, 
                                      RepayCreditAccount_topic, 
                                      LiquidateCreditAccount_topic,
                                      AddCollateral_topic,
                                      IncreaseBorrowedAmount_topic,                                  
                                 ]],
                    from_block,
                    'latest')
    print(datetime.utcnow(), 'get_logs from CM', CM)
    df_events = df_events.append(logs, ignore_index = True)    

print(datetime.utcnow(), 'number of events=', len(df_events))
if len(df_events)>0:
    df_events = df_events.rename(columns={'address': 'CM'})
    df_events['CM_Token'] = df_events['CM'].apply(lambda x: cm_dict[x]['symbol'])
    df_events['blockHash'] = df_events['blockHash'].apply(lambda x: x.hex())
    df_events['transactionHash'] = df_events['transactionHash'].apply(lambda x: x.hex())
    df_events['Borrower'] = df_events.apply(lambda x: x['args']['to'] 
                                                  if x['event'] in ['CloseCreditAccount','CloseCreditAccount'] 
                                                  else x['args']['onBehalfOf'] if x['event'] in ['OpenCreditAccount','AddCollateral']
                                                  else x['args']['borrower'] if x['event'] in ['IncreaseBorrowedAmount']
                                                  else x['args']['owner'] if x['event'] in ['LiquidateCreditAccount']
                                                  else None
                                        ,axis=1)
    df_events['Liquidator'] = df_events.apply(lambda x: x['args']['liquidator'] if x['event'] in ['LiquidateCreditAccount']
                                                  else None
                                        ,axis=1)


    print(datetime.utcnow(), 'pull event time begin')

    df_events['blockTimestamp'] = df_events['blockNumber'].apply(lambda x: datetime.utcfromtimestamp(w3_eth.eth.getBlock(x)['timestamp']))
    df_events['args'] = df_events['args'].apply(str)
    df_events.drop(columns = 'logIndex')
    df_events = df_events.sort_values('blockTimestamp', ignore_index = True)

    print(datetime.utcnow(), 'pull event timestamp end')

    df_events

    pandas_gbq.to_gbq(df_events, 
                      'gearbox.account_events',
                      project_id=gcp_project_id,
                      if_exists = ('replace' if from_block==0 else 'append'), 
                      progress_bar = False)
    print(datetime.utcnow(), 'gearbox.account_events, insert done')
else:
    print('No new events since block', from_block)
    

2022-01-08 03:50:57.356756 from_block= 13956971
2022-01-08 03:50:57.588388 get_logs from CM 0x777E23A2AcB2fCbB35f6ccF98272d03C722Ba6EB
2022-01-08 03:50:57.812988 get_logs from CM 0x2664cc24CBAd28749B3Dd6fC97A6B402484De527
2022-01-08 03:50:58.035934 get_logs from CM 0x968f9a68a98819E2e6Bb910466e191A7b6cf02F0
2022-01-08 03:50:58.257046 get_logs from CM 0xC38478B0A4bAFE964C3526EEFF534d70E1E09017
2022-01-08 03:50:58.258058 number of events= 2
2022-01-08 03:50:58.265914 pull event time begin
2022-01-08 03:50:58.810722 pull event timestamp end
2022-01-08 03:51:03.512902 gearbox.account_events, insert done


**Data Studio Report**
https://datastudio.google.com/reporting/a95186ae-29b4-4d72-8807-612bb5f54dd0